In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import polars as pl
from datasets import load_dataset
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def fast_clean(text: str) -> str:
    text = text.lower()
    text = re.sub(r"http\S+|[^a-zA-Z\s]", "", text)
    tokens = re.findall(r'\b\w+\b', text)
    return " ".join([t for t in tokens if t not in stop_words])

In [ ]:
dataset = load_dataset("artem9k/ai-text-detection-pile")["train"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
seen, texts, labels = set(), [], []
for ex in dataset:
    if ex["text"] and ex["text"].strip() and ex["text"] not in seen:
        seen.add(ex["text"])
        texts.append(ex["text"])
        labels.append(1 if ex["source"] == "ai" else 0)

In [ ]:
df = pl.DataFrame({"text": texts, "label": labels})

In [ ]:
cleaned = [fast_clean(t) for t in df["text"]]
df = df.with_columns(pl.Series(name="cleaned_text", values=cleaned))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_val, y_train, y_val = train_test_split(
    df["cleaned_text"].to_list(), df["label"].to_list(),
    test_size=0.2, random_state=42
)

vectorizer = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    ngram_range=(1, 1),
    sublinear_tf=True,
    dtype='float32'
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:2043: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  warnings.warn(


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
lr = LogisticRegression(max_iter=1000, n_jobs=-1)
lr.fit(X_train_tfidf, y_train)
print("Logistic Regression\n", classification_report(y_val, lr.predict(X_val_tfidf)))


Logistic Regression
               precision    recall  f1-score   support

           0       0.93      0.94      0.94    204218
           1       0.83      0.81      0.82     72951

    accuracy                           0.91    277169
   macro avg       0.88      0.88      0.88    277169
weighted avg       0.91      0.91      0.91    277169



In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf.fit(X_train_tfidf, y_train)
print("RF\n", classification_report(y_val, rf.predict(X_val_tfidf)))

RF
               precision    recall  f1-score   support

           0       0.89      0.94      0.92    204218
           1       0.81      0.68      0.74     72951

    accuracy                           0.88    277169
   macro avg       0.85      0.81      0.83    277169
weighted avg       0.87      0.88      0.87    277169



In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
xgb.fit(X_train_tfidf, y_train)
print("XGBoost\n", classification_report(y_val, xgb.predict(X_val_tfidf)))

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:46:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost
               precision    recall  f1-score   support

           0       0.89      0.93      0.91    204218
           1       0.79      0.69      0.74     72951

    accuracy                           0.87    277169
   macro avg       0.84      0.81      0.83    277169
weighted avg       0.87      0.87      0.87    277169



In [ ]:
! pip install scikit-learn-intelex polars datasets scikit-learn xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 MB 4.8 MB/s eta 0:00:00


In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [ ]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer(norm='l2')
X_train_norm = scaler.fit_transform(X_train_tfidf)
X_val_norm = scaler.transform(X_val_tfidf)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

svm = SVC(kernel='rbf', C=1.0, probability=True)
svm.fit(X_train_norm, y_train)

y_pred_svm = svm.predict(X_val_norm)
print("Intel-Optimized SVM\n", classification_report(y_val, y_pred_svm))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

Intel-Optimized SVM
               precision    recall  f1-score   support

           0       0.95      0.96      0.96    204218
           1       0.88      0.87      0.88     72951

    accuracy                           0.94    277169
   macro avg       0.92      0.92      0.92    277169
weighted avg       0.94      0.94      0.94    277169

